* Drop
* fill
* Replace
* Trunc
* Explode--- explode_outer
* Flatten
* Format_string
* JSON functions

In [1]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql.functions import udf,col,length,expr,lit,lower,sumDistinct,sum,round,count,countDistinct,max,min,avg
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Train_hard").getOrCreate()
from pyspark.sql.window import Window
from pyspark.sql.window import *
from pyspark.sql.functions import *
from pyspark.sql.types import MapType,StringType,StructField

# NA_functions

### Drop

In [6]:
data=[("Alice",80,10),("Bob",None,5),("Tom",50,50),(None,None,None),("Robert",30,35),("Jin",None,None)]
schema="name string, Age int, Height int"
df=spark.createDataFrame(data=data,schema=schema)
df.show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|  null|null|  null|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



In [7]:
df.dropna().show()

+------+---+------+
|  name|Age|Height|
+------+---+------+
| Alice| 80|    10|
|   Tom| 50|    50|
|Robert| 30|    35|
+------+---+------+



### How in Drop
* by defaut its how="any"
* if we give how="all", row having all null value will be dropped

In [8]:
df.dropna(how='all').show()
# only row having all elemnt as null is dropped

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



### Thres
* thres=1, means if having less than 1 not-null value, row 4 have all null value so it's dropped

In [9]:
df.dropna(thresh=1).show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



In [10]:
df.dropna(thresh=2).show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|Robert|  30|    35|
+------+----+------+



In [11]:
df.dropna(thresh=3).show()

+------+---+------+
|  name|Age|Height|
+------+---+------+
| Alice| 80|    10|
|   Tom| 50|    50|
|Robert| 30|    35|
+------+---+------+



### Using-- Subset

In [13]:
df.dropna(subset="Height").show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|Robert|  30|    35|
+------+----+------+



In [14]:
df.dropna(subset=["Age","Height"]).show()

+------+---+------+
|  name|Age|Height|
+------+---+------+
| Alice| 80|    10|
|   Tom| 50|    50|
|Robert| 30|    35|
+------+---+------+



## Fill
* we pass 55 as int so it filled only int columns string column remain as it is

In [16]:
df.fillna(55).show()

+------+---+------+
|  name|Age|Height|
+------+---+------+
| Alice| 80|    10|
|   Bob| 55|     5|
|   Tom| 50|    50|
|  null| 55|    55|
|Robert| 30|    35|
|   Jin| 55|    55|
+------+---+------+



In [17]:
df.fillna("king").show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|  king|null|  null|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



### Another way using multiple variables

In [25]:
df.fillna({"name":"Rocky","Age":121}).show()

+------+---+------+
|  name|Age|Height|
+------+---+------+
| Alice| 80|    10|
|   Bob|121|     5|
|   Tom| 50|    50|
| Rocky|121|  null|
|Robert| 30|    35|
|   Jin|121|  null|
+------+---+------+



### Replace

In [26]:
df.replace(50,155).show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom| 155|   155|
|  null|null|  null|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



In [27]:
df.replace(50,155,subset="Age").show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom| 155|    50|
|  null|null|  null|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



In [29]:
df.replace({"Alice":"Alex","Bob":"Bobby"},subset="name").show()

+------+----+------+
|  name| Age|Height|
+------+----+------+
|  Alex|  80|    10|
| Bobby|null|     5|
|   Tom|  50|    50|
|  null|null|  null|
|Robert|  30|    35|
|   Jin|null|  null|
+------+----+------+



## ABS

In [31]:
df1=spark.createDataFrame([(-5,0),(1,3),(7,9)],("col1","col2"))
df1.show()

+----+----+
|col1|col2|
+----+----+
|  -5|   0|
|   1|   3|
|   7|   9|
+----+----+



## Trunc--can be applied to timestamp or date column

In [32]:
df=spark.read.load('orders/part-00000',format="csv",schema='order_id int,order_date timestamp,order_customer_id int, \
                   order_status string')
df.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [42]:
df.select("order_date",trunc(df.order_date,"year")).distinct().show(5)

+-------------------+-----------------------+
|         order_date|trunc(order_date, year)|
+-------------------+-----------------------+
|2014-01-30 00:00:00|             2014-01-01|
|2013-08-30 00:00:00|             2013-01-01|
|2014-04-28 00:00:00|             2014-01-01|
|2013-09-03 00:00:00|             2013-01-01|
|2013-10-25 00:00:00|             2013-01-01|
+-------------------+-----------------------+
only showing top 5 rows



In [43]:
df.select("order_date",trunc(df.order_date,"mon")).distinct().show(5)

+-------------------+----------------------+
|         order_date|trunc(order_date, mon)|
+-------------------+----------------------+
|2014-01-30 00:00:00|            2014-01-01|
|2013-12-16 00:00:00|            2013-12-01|
|2013-12-26 00:00:00|            2013-12-01|
|2014-03-12 00:00:00|            2014-03-01|
|2013-12-18 00:00:00|            2013-12-01|
+-------------------+----------------------+
only showing top 5 rows



## Explode

In [30]:
data=[("Alicia",["Java","Scala"],{"Hair":"Black","Eye":"Brown"}),\
     ("Robert",["Spark","Java"],{"Hair":"Brown","Eye":None}),\
     ("Mike",["CSharp",""],{"Hair":"Red","Eye":""}),\
     ("John",None,None),\
     ("Jeff",["1","2"],{})]
schema=["Empname","Languages","Properties"]
df=spark.createDataFrame(data,schema)
df.show(truncate=False)

+-------+-------------+-----------------------------+
|Empname|Languages    |Properties                   |
+-------+-------------+-----------------------------+
|Alicia |[Java, Scala]|{Eye -> Brown, Hair -> Black}|
|Robert |[Spark, Java]|{Eye -> null, Hair -> Brown} |
|Mike   |[CSharp, ]   |{Eye -> , Hair -> Red}       |
|John   |null         |null                         |
|Jeff   |[1, 2]       |{}                           |
+-------+-------------+-----------------------------+



In [9]:
df.select(df.Empname,explode(df.Languages)).show()

+-------+------+
|Empname|   col|
+-------+------+
| Alicia|  Java|
| Alicia| Scala|
| Robert| Spark|
| Robert|  Java|
|   Mike|CSharp|
|   Mike|      |
|   Jeff|     1|
|   Jeff|     2|
+-------+------+



## NOTE: HERE
* John have null record in the array so its ignored in explode

## Explode on Map column

In [10]:
df.printSchema()

root
 |-- Empname: string (nullable = true)
 |-- Languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [11]:
df.select(df.Empname,explode(df.Properties)).show()

+-------+----+-----+
|Empname| key|value|
+-------+----+-----+
| Alicia| Eye|Brown|
| Alicia|Hair|Black|
| Robert| Eye| null|
| Robert|Hair|Brown|
|   Mike| Eye|     |
|   Mike|Hair|  Red|
+-------+----+-----+



## Explode_Outer-- 
* Unlike explode it returns null values as well

In [12]:
df.select(df.Empname,explode_outer(df.Languages)).show()

+-------+------+
|Empname|   col|
+-------+------+
| Alicia|  Java|
| Alicia| Scala|
| Robert| Spark|
| Robert|  Java|
|   Mike|CSharp|
|   Mike|      |
|   John|  null|
|   Jeff|     1|
|   Jeff|     2|
+-------+------+



In [13]:
df.select(df.Empname,explode_outer(df.Properties)).show()

+-------+----+-----+
|Empname| key|value|
+-------+----+-----+
| Alicia| Eye|Brown|
| Alicia|Hair|Black|
| Robert| Eye| null|
| Robert|Hair|Brown|
|   Mike| Eye|     |
|   Mike|Hair|  Red|
|   John|null| null|
|   Jeff|null| null|
+-------+----+-----+



## Posexplode
* Display position as well, IGnores null value

In [14]:
df.select(df.Empname,posexplode(df.Languages)).show()

+-------+---+------+
|Empname|pos|   col|
+-------+---+------+
| Alicia|  0|  Java|
| Alicia|  1| Scala|
| Robert|  0| Spark|
| Robert|  1|  Java|
|   Mike|  0|CSharp|
|   Mike|  1|      |
|   Jeff|  0|     1|
|   Jeff|  1|     2|
+-------+---+------+



In [15]:
df.select(df.Empname,posexplode(df.Properties)).show()

+-------+---+----+-----+
|Empname|pos| key|value|
+-------+---+----+-----+
| Alicia|  0| Eye|Brown|
| Alicia|  1|Hair|Black|
| Robert|  0| Eye| null|
| Robert|  1|Hair|Brown|
|   Mike|  0| Eye|     |
|   Mike|  1|Hair|  Red|
+-------+---+----+-----+



## posexplode_outer
* Reports NUll as well

In [16]:
df.select(df.Empname,posexplode_outer(df.Languages)).show()

+-------+----+------+
|Empname| pos|   col|
+-------+----+------+
| Alicia|   0|  Java|
| Alicia|   1| Scala|
| Robert|   0| Spark|
| Robert|   1|  Java|
|   Mike|   0|CSharp|
|   Mike|   1|      |
|   John|null|  null|
|   Jeff|   0|     1|
|   Jeff|   1|     2|
+-------+----+------+



In [17]:
df.select(df.Empname,posexplode_outer(df.Properties)).show()

+-------+----+----+-----+
|Empname| pos| key|value|
+-------+----+----+-----+
| Alicia|   0| Eye|Brown|
| Alicia|   1|Hair|Black|
| Robert|   0| Eye| null|
| Robert|   1|Hair|Brown|
|   Mike|   0| Eye|     |
|   Mike|   1|Hair|  Red|
|   John|null|null| null|
|   Jeff|null|null| null|
+-------+----+----+-----+



## Flatten--Convert an array of array into a single Array column

In [18]:
data=[("Alicia",[["Java"],["Scala"],["Python"]]),\
     ("Robert",[[None],["Java"],["Hadoop"]])]
schema=["Empname","ArrayOfArrays"]
df=spark.createDataFrame(data,schema)
df.show(truncate=False)

+-------+---------------------------+
|Empname|ArrayOfArrays              |
+-------+---------------------------+
|Alicia |[[Java], [Scala], [Python]]|
|Robert |[[null], [Java], [Hadoop]] |
+-------+---------------------------+



In [21]:
df.select(col("*"),flatten(df.ArrayOfArrays)).show(truncate=False)

+-------+---------------------------+----------------------+
|Empname|ArrayOfArrays              |flatten(ArrayOfArrays)|
+-------+---------------------------+----------------------+
|Alicia |[[Java], [Scala], [Python]]|[Java, Scala, Python] |
|Robert |[[null], [Java], [Hadoop]] |[null, Java, Hadoop]  |
+-------+---------------------------+----------------------+



### Faltten when we pass not a nested list

In [22]:
data=[("Alicia",[[1],[2]]),\
     ("Robert",[None,[55]])]
schema=["Empname","ArrayOfArrays"]
df=spark.createDataFrame(data,schema)
df.show(truncate=False)

+-------+-------------+
|Empname|ArrayOfArrays|
+-------+-------------+
|Alicia |[[1], [2]]   |
|Robert |[null, [55]] |
+-------+-------------+



In [23]:
df.select(col("*"),flatten(df.ArrayOfArrays)).show(truncate=False)

+-------+-------------+----------------------+
|Empname|ArrayOfArrays|flatten(ArrayOfArrays)|
+-------+-------------+----------------------+
|Alicia |[[1], [2]]   |[1, 2]                |
|Robert |[null, [55]] |null                  |
+-------+-------------+----------------------+



### as in case of robert-
* There is no nested list so null is returned

In [27]:
data=[("Alicia",[[1],[2]]),\
     ("Robert",[[None],[55]])]
schema=["Empname","ArrayOfArrays"]
df=spark.createDataFrame(data,schema)
df.show(truncate=False)

+-------+--------------+
|Empname|ArrayOfArrays |
+-------+--------------+
|Alicia |[[1], [2]]    |
|Robert |[[null], [55]]|
+-------+--------------+



In [28]:
df.select(col("*"),flatten(df.ArrayOfArrays)).show(truncate=False)

+-------+--------------+----------------------+
|Empname|ArrayOfArrays |flatten(ArrayOfArrays)|
+-------+--------------+----------------------+
|Alicia |[[1], [2]]    |[1, 2]                |
|Robert |[[null], [55]]|[null, 55]            |
+-------+--------------+----------------------+



## Note
* we can apply multiple flatten functions as well

In [29]:
df.select(col("ArrayOfArrays"),flatten(df.ArrayOfArrays),flatten(df.ArrayOfArrays)).show()

+--------------+----------------------+----------------------+
| ArrayOfArrays|flatten(ArrayOfArrays)|flatten(ArrayOfArrays)|
+--------------+----------------------+----------------------+
|    [[1], [2]]|                [1, 2]|                [1, 2]|
|[[null], [55]]|            [null, 55]|            [null, 55]|
+--------------+----------------------+----------------------+



# NOTE
* If we apply multiple explote it throws error

In [31]:
df.show()

+-------+-------------+--------------------+
|Empname|    Languages|          Properties|
+-------+-------------+--------------------+
| Alicia|[Java, Scala]|{Eye -> Brown, Ha...|
| Robert|[Spark, Java]|{Eye -> null, Hai...|
|   Mike|   [CSharp, ]|{Eye -> , Hair ->...|
|   John|         null|                null|
|   Jeff|       [1, 2]|                  {}|
+-------+-------------+--------------------+



In [33]:
df.select(explode(df.Languages),explode(df.Properties)).show()

AnalysisException: Only one generator allowed per select clause but found 2: explode(Languages), explode(Properties)

# Formatting Functions

In [2]:
df=spark.read.load('Order_items/part-00000',format="csv",schema='order_id int,order_item_order_id int,order_item_product_id int, \
                   quantity int, subtotal float, price float')
df.show(3)

+--------+-------------------+---------------------+--------+--------+------+
|order_id|order_item_order_id|order_item_product_id|quantity|subtotal| price|
+--------+-------------------+---------------------+--------+--------+------+
|       1|                  1|                  957|       1|  299.98|299.98|
|       2|                  2|                 1073|       1|  199.99|199.99|
|       3|                  2|                  502|       5|   250.0|  50.0|
+--------+-------------------+---------------------+--------+--------+------+
only showing top 3 rows



In [4]:
df1=spark.createDataFrame([(5,"Hello")],("a","b"))
df1.show()

+---+-----+
|  a|    b|
+---+-----+
|  5|Hello|
+---+-----+



## Format_string
* print the output like print statement in python

In [6]:
df1.select(col("*"),format_string("%d %s",df1.a,df1.b)).show()

+---+-----+--------------------------+
|  a|    b|format_string(%d %s, a, b)|
+---+-----+--------------------------+
|  5|Hello|                   5 Hello|
+---+-----+--------------------------+



## JSON Functions

In [2]:
data=[(1,""" {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}""")]
df_map=spark.createDataFrame(data,("ID","Value"))
df_map.show(truncate=False)

+---+-------------------------------------------------------------------------+
|ID |Value                                                                    |
+---+-------------------------------------------------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|
+---+-------------------------------------------------------------------------+



In [5]:
df_map.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)



In [4]:
data=[(1,""" [1,2,3]""")]
df_arr=spark.createDataFrame(data,("ID","Value"))
df_arr.show(truncate=False)

+---+--------+
|ID |Value   |
+---+--------+
|1  | [1,2,3]|
+---+--------+



In [6]:
df_arr.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)



In [7]:
data=[(1,""" {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}""")]
df_struct=spark.createDataFrame(data,("ID","Value"))
df_struct.show(truncate=False)

+---+-------------------------------------------------------------------------+
|ID |Value                                                                    |
+---+-------------------------------------------------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|
+---+-------------------------------------------------------------------------+



# JSON

### From_json
* Convert a json string into a MapType or Struct Type or ArrayType

In [14]:
df_map.show()

+---+--------------------+
| ID|               Value|
+---+--------------------+
|  1| {"Zipcode":85016...|
+---+--------------------+



In [8]:
schema= MapType(StringType(),StringType())
df_map.withColumn("Map_column",from_json(df_map.Value,schema)).show(truncate=False)

+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
|ID |Value                                                                    |Map_column                                                               |
+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|{Zipcode -> 85016, Zipcodetype -> STANDARD, City -> Phoenix, State -> AZ}|
+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+



In [10]:
df_nw=df_map.withColumn("Map_column",from_json(df_map.Value,schema))
df_nw.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)
 |-- Map_column: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [24]:
df_nw.select(df_nw.Map_column["Zipcode"]).show()

+-------------------+
|Map_column[Zipcode]|
+-------------------+
|              85016|
+-------------------+



## Converting Json into Array TYpe

In [48]:
df_arr.show()

+---+--------+
| ID|   Value|
+---+--------+
|  1| [1,2,3]|
+---+--------+



In [21]:
schema=ArrayType(IntegerType())
df_array=df_arr.withColumn("Arr_column",from_json(df_arr.Value,schema))
df_array.show()

+---+--------+----------+
| ID|   Value|Arr_column|
+---+--------+----------+
|  1| [1,2,3]| [1, 2, 3]|
+---+--------+----------+



In [22]:
df_array.select(df_array.Arr_column[0]).show()

+-------------+
|Arr_column[0]|
+-------------+
|            1|
+-------------+



In [30]:
df_array.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)
 |-- Arr_column: array (nullable = true)
 |    |-- element: integer (containsNull = true)



## From json to structType

In [36]:
df_struct.show(truncate=False)

+---+-------------------------------------------------------------------------+
|ID |Value                                                                    |
+---+-------------------------------------------------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|
+---+-------------------------------------------------------------------------+



In [23]:
schema=StructType([StructField("Zipcode",IntegerType()),StructField("Zipcodetype",StringType()),\
                   StructField("City",StringType()),StructField("State",StringType())])

In [24]:
df_struct.withColumn("Struct_column",from_json(df_struct.Value,schema)).show(truncate=False)

+---+-------------------------------------------------------------------------+------------------------------+
|ID |Value                                                                    |Struct_column                 |
+---+-------------------------------------------------------------------------+------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|{85016, STANDARD, Phoenix, AZ}|
+---+-------------------------------------------------------------------------+------------------------------+



In [25]:
df_struct_nw=df_struct.withColumn("Struct_column",from_json(df_struct.Value,schema))
df_struct_nw.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)
 |-- Struct_column: struct (nullable = true)
 |    |-- Zipcode: integer (nullable = true)
 |    |-- Zipcodetype: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- State: string (nullable = true)



In [47]:
df_struct_nw.select(df_struct_nw.Struct_column.Zipcode).show()

+---------------------+
|Struct_column.Zipcode|
+---------------------+
|                85016|
+---------------------+



## TO_json()
* Converts map typr into String type

In [13]:
df_nw.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)
 |-- Map_column: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [16]:
df_nw.select(df_nw.Map_column,to_json(df_nw.Map_column)).printSchema()

root
 |-- Map_column: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- to_json(Map_column): string (nullable = true)



#### Similarly convert array type to string

In [26]:
df_array.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)
 |-- Arr_column: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [27]:
df_array.select(df_array.Arr_column,to_json(df_array.Arr_column)).printSchema()

root
 |-- Arr_column: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- to_json(Arr_column): string (nullable = true)



#### Similarly convert Struct type to string

In [30]:
df_struct_nw.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Value: string (nullable = true)
 |-- Struct_column: struct (nullable = true)
 |    |-- Zipcode: integer (nullable = true)
 |    |-- Zipcodetype: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- State: string (nullable = true)



In [31]:
df_struct_nw.select(df_struct_nw.Struct_column,to_json(df_struct_nw.Struct_column)).printSchema()

root
 |-- Struct_column: struct (nullable = true)
 |    |-- Zipcode: integer (nullable = true)
 |    |-- Zipcodetype: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- State: string (nullable = true)
 |-- to_json(Struct_column): string (nullable = true)



## json_tuple()
* Extract elemnt from json string column and create the result as a new column

In [33]:
df_map.show(truncate=False)

+---+-------------------------------------------------------------------------+
|ID |Value                                                                    |
+---+-------------------------------------------------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|
+---+-------------------------------------------------------------------------+



* Uisng on;y zipcode,city and state as the coumns

In [37]:
df_map.select(json_tuple(df_map.Value,"Zipcode","City","State")).toDF("ZIP","SEHER","RAJYE").show()

+-----+-------+-----+
|  ZIP|  SEHER|RAJYE|
+-----+-------+-----+
|85016|Phoenix|   AZ|
+-----+-------+-----+



## get_json_object()

In [41]:
df_map.select(col("*")).show(truncate=False)

+---+-------------------------------------------------------------------------+
|ID |Value                                                                    |
+---+-------------------------------------------------------------------------+
|1  | {"Zipcode":85016,"Zipcodetype":"STANDARD","City":"Phoenix","State":"AZ"}|
+---+-------------------------------------------------------------------------+



In [42]:
df_map.select(col("id"),get_json_object(col("Value"),"$.Zipcode").alias("ZIP")).show()

+---+-----+
| id|  ZIP|
+---+-----+
|  1|85016|
+---+-----+

